In [1]:
import sklearn.datasets
import re
from sklearn.cross_validation import train_test_split
import numpy as np

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def clearstring(string):
    string = re.sub('[^A-Za-z0-9 ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    string = ' '.join(string)
    return string

def separate_dataset(trainset):
    datastring = []
    datatarget = []
    for i in range(len(trainset.data)):
        data_ = trainset.data[i].split('\n')
        data_ = list(filter(None, data_))
        for n in range(len(data_)):
            data_[n] = clearstring(data_[n])
        datastring += data_
        for n in range(len(data_)):
            datatarget.append(trainset.target[i])
    return datastring, datatarget

In [3]:
trainset = sklearn.datasets.load_files(container_path = 'local', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['kerajaan', 'pembangkang']
201
201


In [4]:
vocabulary = list(set(' '.join(trainset.data).split()))
len(vocabulary)

1737

In [5]:
# calculate IDF
idf = {}
for i in vocabulary:
    idf[i] = 0
    for k in trainset.data:
        if i in k.split():
            idf[i] += 1
    idf[i] = np.log(idf[i] / len(trainset.data))

# calculate TF
X = np.zeros((len(trainset.data),len(vocabulary)))
for no, i in enumerate(trainset.data):
    for text in i.split():
        X[no, vocabulary.index(text)] += 1
    for text in i.split():
        # calculate TF * IDF
        X[no, vocabulary.index(text)] = X[no, vocabulary.index(text)] * idf[text]

In [6]:
train_X, test_X, train_Y, test_Y = train_test_split(X, trainset.target, test_size = 0.2)

In [7]:
class MultinomialNB:
    def __init__(self, epsilon):
        self.EPSILON = epsilon

    def fit(self, X, y):
        count_sample = X.shape[0]
        separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]
        self.class_log_prior_ = [np.log(len(i) / count_sample + self.EPSILON) for i in separated]
        count = np.array([np.array(i).sum(axis=0) for i in separated])
        self.feature_log_prob_ = np.log(count / count.sum(axis=1)[np.newaxis].T + self.EPSILON)

    def predict_log_proba(self, X):
        return [(self.feature_log_prob_ * x).sum(axis=1) + self.class_log_prior_ for x in X]

    def predict(self, X):
        return np.argmax(self.predict_log_proba(X), axis=1)

In [8]:
multinomial_bayes = MultinomialNB(1e-8)
multinomial_bayes.fit(train_X, train_Y)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in log
  # Remove the CWD from sys.path while we load stuff.


## accuracy training

In [9]:
np.mean(train_Y == multinomial_bayes.predict(train_X))

0.49375000000000002

## accuracy testing

In [10]:
np.mean(test_Y == multinomial_bayes.predict(test_X))

0.51219512195121952